In [2]:
#https://blog.csdn.net/weixin_42966923/article/details/131530823
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
%matplotlib auto

# 模型定义
net= tf.keras.Sequential([tf.keras.layers.Input(shape=(2,)),
                          tf.keras.layers.Dense(256,activation='tanh'),
                          tf.keras.layers.Dense(256,activation='tanh'),
                          tf.keras.layers.Dense(256,activation='tanh'),
                          tf.keras.layers.Dense(1)])


# 损失函数
mse = tf.keras.losses.MeanSquaredError()
# 优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
# 打包
net.compile(optimizer,loss=mse)

plt.ion() #动态图
fig = plt.figure(figsize=(6,5))
iterations=200

#初始化常量
W= 0.01 /3.1415926
t_bc_zeros = tf.constant(np.zeros((2000,1)),dtype=tf.float32)
x_in_pos_one = tf.constant(np.ones((2000,1)),dtype=tf.float32)
x_in_neg_one = tf.constant(-np.ones((2000,1)),dtype=tf.float32)
u_in_zeros = tf.constant(np.zeros((2000,1)),dtype=tf.float32)

for epoch in range(iterations):
    with tf.GradientTape()as tape:
       #初始化变量
       t_var = tf.Variable(tf.random.uniform((2000, 1),dtype=tf.float32, minval=0.0, maxval=1.0))
       x_var = tf.Variable(tf.random.uniform((2000,1),dtype=tf.float32, minval=-1.0, maxval=1.0))
       #求一阶、二阶偏导
       with tf.GradientTape(persistent=True) as tape_xx:
          with tf.GradientTape(persistent=True) as tape_x:
             u_hat = net(tf.concat([t_var,x_var], axis=1))
          du_dt = tape_x.gradient(u_hat, t_var)
          du_dx = tape_x.gradient(u_hat, x_var)
       du_dxx= tape_xx.gradient(du_dx, x_var)

       # eq(1)
       eq1_1= du_dt + u_hat * du_dx -(0.01/3.14)* du_dxx
       mse_1 = mse(eq1_1,u_in_zeros)
       # eq(2)
       eq2_1 = net(tf.concat([t_bc_zeros, x_var], axis=1))
       eq2_2 = -tf.sin(3.1415926 * x_var)
       mse_2 = mse(eq2_1,eq2_2)
       # eq(3)
       eq3_1 = net(tf.concat([t_var,x_in_pos_one],axis=1))
       mse_3 = mse(eq3_1,u_in_zeros)
       # eq(4)
       eq4_1 = net(tf.concat([t_var,x_in_neg_one], axis=1))
       mse_4 = mse(eq4_1,u_in_zeros)
     
       loss = mse_1 + mse_2 + mse_3 + mse_4
  
    gradients =tape.gradient(loss, net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))

    if(epoch+1)% (iterations/2) == 0:
      t=np.linspace(0,1,100)
      x=np.linspace(-1,1,256)
      ms_t,ms_x=np.meshgrid(t,x)
      x= np.ravel(ms_x).reshape(-1,1)
      t=np.ravel(ms_t).reshape(-1,1)
      pt_u= net(tf.concat([t, x],axis=1))
      u= pt_u.numpy().reshape(ms_t.shape)

      fig.clf() # 清空当前Figure对象
      ax = fig.add_subplot(111,projection='3d')
      ax.set_zlim([-1,1])
      #在图中添加文字
      ax.text(0,6,1,"epoch:%d"%(epoch+1),color='black')
      ax.plot_surface(ms_t, ms_x, u, cmap=cm.RdYlBu_r, edgecolor='blue', linewidth=0.0003, antialiased=True)
      ax.set_xlabel('t')
      ax.set_ylabel('x')
      ax.set_zlabel('u')
      plt.pause(0.1)

    if(epoch+1==iterations) :
      Output=tf.concat([x,t,u], axis=1)
      df=pd.DataFrame(Output)
      df.to_excel('C:/Users/pengt/Desktop/Output.xlsx',index=False)
plt.show()

Using matplotlib backend: <object object at 0x000001F3725A49C0>


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Dimension 0 in both shapes must be equal: shape[0] = [25600,1] vs. shape[2] = [256,100] [Op:ConcatV2] name: concat